In [1]:
# URL: http://scikit-learn.org/stable/modules/cross_validation.html

In [2]:
"""
Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: 
a model that would just repeat the labels of the samples that it has just seen would have a perfect score but
would fail to predict anything useful on yet-unseen data. This situation is called overfitting. To avoid it, 
it is common practice when performing a (supervised) machine learning experiment to hold out part of the available 
data as a test set X_test, y_test. Note that the word “experiment” is not intended to denote academic use only, 
because even in commercial settings machine learning usually starts out experimentally.

In scikit-learn a random split into training and test sets can be quickly computed with the train_test_split 
helper function. Let’s load the iris data set to fit a linear support vector machine on it:
"""
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

iris = datasets.load_iris()
iris.data.shape, iris.target.shape

((150, 4), (150,))

In [3]:
"""
We can now quickly sample a training set while holding out 40% of the data for testing (evaluating) our 
classifier:
"""
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.4, random_state = 0)
X_train.shape, y_train.shape

((90, 4), (90,))

In [4]:
X_test.shape, y_test.shape

((60, 4), (60,))

In [5]:
clf = svm.SVC(kernel = 'linear', C = 1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.96666666666666667

In [6]:
"""
When evaluating different settings (“hyperparameters”) for estimators, such as the C setting that must be 
manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can 
be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into 
the model and evaluation metrics no longer report on generalization performance. To solve this problem, 
yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the 
training set, after which evaluation is done on the validation set, and when the experiment seems to be 
successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which 
can be used for learning the model, and the results can depend on a particular random choice for the pair of 
(train, validation) sets.

A solution to this problem is a procedure called cross-validation (CV for short). A test set should still 
be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, 
called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but 
generally follow the same principles). The following procedure is followed for each of the k “folds”:
    - A model is trained using k-1 of the folds as training data;
    - the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to 
    compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the 
loop. This approach can be computationally expensive, but does not waste too much data (as it is the case when 
fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number 
of samples is very small.
"""
print('')

## Computing cross-validated metrics

In [7]:
"""
The simplest way to use cross-validation is to call the cross_val_score helper function on the estimator 
and the dataset.

The following example demonstrates how to estimate the accuracy of a linear kernel support vector machine on
the iris dataset by splitting the data, fitting a model and computing the score 5 consecutive times 
(with different splits each time):
"""
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel = 'linear', C = 1)
scores = cross_val_score(clf, iris.data, iris.target, cv = 5)
scores

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

In [8]:
"""
The mean score and the 95% confidence interval of the score estimate are hence given by:
"""
print('Accuracy %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2 ))

Accuracy 0.98 (+/- 0.03)


In [9]:
"""
By default, the score computed at each CV iteration is the score method of the estimator. It is possible to 
change this by using the scoring parameter:
"""
from sklearn import metrics
scores = cross_val_score(clf, iris.data, iris.target, cv = 5, scoring = 'f1_macro')
scores

array([ 0.96658312,  1.        ,  0.96658312,  0.96658312,  1.        ])

In [10]:
"""
See The scoring parameter: defining model evaluation rules for details
(http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter). In the case of the Iris dataset,
the samples are balanced across target classes hence the accuracy and the F1-score are almost equal.

When the cv argument is an integer, cross_val_score uses the KFold or StratifiedKFold strategies by default,
the latter being used if the estimator derives from ClassifierMixin.

It is also possible to use other cross validation strategies by passing a cross validation iterator instead, 
for instance:
"""
from sklearn.model_selection import ShuffleSplit
n_samples = iris.data.shape[0]
cv = ShuffleSplit(n_splits = 3, test_size = 0.3, random_state = 0)
cross_val_score(clf, iris.data, iris.target, cv = cv )

array([ 0.97777778,  0.97777778,  1.        ])

###### Data transformation with held out data 

In [11]:
"""
Just as it is important to test a predictor on data held-out from training, preprocessing (such as 
standardization, feature selection, etc.) and similar data transformations similarly should be learnt from a 
training set and applied to held-out data for prediction:
"""
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size = 0.4, random_state = 0)
scaler = preprocessing.StandardScaler().fit(X_train, y_train)
X_train_trasformed = scaler.transform(X_train)
clf = svm.SVC(C = 1).fit(X_train_trasformed, y_train)
X_test_trasformed = scaler.transform(X_test)
clf.score(X_test_trasformed, y_test)

0.93333333333333335

In [12]:
"""
A Pipeline makes it easier to compose estimators, providing this behavior under cross-validation:
"""
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C = 1))
cross_val_score(clf, iris.data, iris.target, cv = cv)

array([ 0.97777778,  0.93333333,  0.95555556])

### The cross_validate function and multiple metric evaluation

In [13]:
"""
The cross_validate function differs from cross_val_score in two ways:
    - It allows specifying multiple metrics for evaluation.
    - It returns a dict containing training scores, fit-times and score-times in addition to the test score.

For single metric evaluation, where the scoring parameter is a string, callable or None, the keys will be - 
['test_score', 'fit_time', 'score_time']

And for multiple metric evaluation, the return value is a dict with the following keys - 
['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score is set to True by default. It adds train score keys for all the scorers. If train scores 
are not needed, this should be set to False explicitly.

The multiple metrics can be specified either as a list, tuple or set of predefined scorer names:
"""
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel = 'linear', C = 1, random_state = 0)
scores = cross_validate(clf, iris.data, iris.target, scoring = scoring, cv = 5, return_train_score = False)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [14]:
scores['test_recall_macro']

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

In [15]:
"""
Or as a dict mapping scorer name to a predefined or custom scoring function:
"""
from sklearn.metrics.scorer import make_scorer
scoring = {'prec_macro' : 'precision_macro',
           'rec_micro': make_scorer(recall_score, average = 'macro')}
scores = cross_validate(clf, iris.data, iris.target, scoring = scoring, cv = 5, return_train_score = True)
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_prec_macro',
 'test_rec_micro',
 'train_prec_macro',
 'train_rec_micro']

In [16]:
scores['train_rec_micro']

array([ 0.975     ,  0.975     ,  0.99166667,  0.98333333,  0.98333333])

In [17]:
"""
Here is an example of cross_validate using a single metric:
"""
scores = cross_validate(clf, iris.data, iris.target, scoring = 'precision_macro')
sorted(scores.keys())

['fit_time', 'score_time', 'test_score', 'train_score']

### Obtaining predictions by cross-validation

In [18]:
"""
The function cross_val_predict has a similar interface to cross_val_score, but returns, for each element
in the input, the prediction that was obtained for that element when it was in the test set. Only 
cross-validation strategies that assign all elements to a test set exactly once can be used (otherwise, 
an exception is raised).

These prediction can then be used to evaluate the classifier:
"""
from sklearn.model_selection import cross_val_predict
predicted = cross_val_predict(clf, iris.data, iris.target, cv = 10)
metrics.accuracy_score(iris.target, predicted)

0.97333333333333338

In [19]:
"""
Note that the result of this computation may be slightly different from those obtained using cross_val_score 
as the elements are grouped in different ways.

The available cross validation iterators are introduced in the following section.
"""
print('')